In [ ]:
# default_exp dbn


In [ ]:
# hide
from fastcore.all import *

# Support code for Dynamic Bayes Nets

Some specialized code for Dynamic Bayes Nets.

In [ ]:
# export
import gtsam

Create or amend a DotWriter to be use in show:

In [ ]:
# export
def dbn_writer(writer=None, hints: dict = None, positions: dict = None,
               boxes: set = None, **kwargs):
    """ Create a DotWriter depending on input arguments:
        If writer is supplied, we will add but not overwrite hints or positions.
    """
    if writer is None and hints is None and positions is None and boxes is None:
        return None
    writer = gtsam.DotWriter() if writer is None else writer
    # Copy hints without overwriting
    if hints is not None:
        assert isinstance(hints, dict)
        ph: dict = writer.positionHints
        for key, y in hints.items():
            if key not in ph:
                ph[key] = y
        writer.positionHints = ph
    # Copy positions without overwriting
    if positions is not None:
        assert isinstance(positions, dict)
        kp: dict = writer.variablePositions
        for key, position in positions.items():
            if key not in kp:
                kp[key] = position
        writer.variablePositions = kp
    # Add boxes
    if boxes is not None:
        assert isinstance(boxes, set)
        bx: set = writer.boxes
        for key in boxes:
            bx.add(key)
        writer.boxes = bx
    return writer


def has_positions(writer):
    """Check if writer has positions for engine selection"""
    if writer is None:
        return False
    return len(writer.positionHints) > 0 or len(writer.variablePositions) > 0


In [ ]:
# Check None cases
assert dbn_writer() is None
assert dbn_writer(exact=True) is None

# Check passthrough
writer = gtsam.DotWriter()
test_eq(dbn_writer(writer), writer)
test_eq(has_positions(writer), False)

# Check boxes, and that they don't stomp
writer = dbn_writer(boxes={1, 2})
test_eq(writer.boxes, {1, 2})
writer = dbn_writer(writer, boxes={2, 3})
test_eq(writer.boxes, {1, 2, 3})
test_eq(has_positions(writer), False)

# Check hints, and that they don't stomp
writer = dbn_writer(hints={"A": 2})
test_eq(writer.positionHints, {"A": 2})
writer = dbn_writer(writer, hints={"A": 3})
test_eq(writer.positionHints, {"A": 2})
test_eq(has_positions(writer), True)

# Check positions, and that they don't stomp
key = 123
writer = dbn_writer(positions={key: (2, 0)})
test_eq(len(writer.variablePositions), 1)
writer = dbn_writer(writer, positions={key: (3, 0)})
test_eq(len(writer.variablePositions), 1)
test_eq(writer.variablePositions[key], (2, 0))
test_eq(has_positions(writer), True)

